<h2>Summit Take Home Challenge</h2>
<h3>David Shirley</h3>

With four reference files, produce a single file that contains the following fields:
* A unique identifier
* Last name
* First name
* School site
* Race/Ethnicity
* Gender
* Whether the student has an IEP

Submit the following:
* Your output file
* A brief explanation of what you chose as the unique identifier and why
* A brief explanation of any problems you noticed in the input data
* The methodology you used to do the task (e.g. a GitHub link, spreadsheet file, etc.) You may use any tool you wish, however the use of a programming language (such as Python) is encouraged.

<h3>Summary of Results</h3>

* Included in my email, find output.tsv, which contains all fields requested

* I chose to concatenate Local ID and Student ID to generate a unique ID for each student. It appeared California schools were reusing Local IDs while Washington schools were reusing Student IDs, so neither field alone would generate a unique identifier after merging the two sets of data.

* Washington student race data is largely missing from this dataset, and there are a few incomplete records among Grade, Gender, and IEP status.

* This work was completed exclusively with Pandas. This GitHub repository and this Juptyer Notebook serve as a record of my work for future reference, and all work was completed in its own IDE to ensure future replicability.

The data used has not been uploaded to this repository.

In [1]:
# Loading the data into Pandas

import pandas as pd

ca_enroll = pd.read_csv("ca_roster.tsv", sep="\t")
wa_enroll = pd.read_csv("wa_roster.tsv", sep="\t")

ca_dem = pd.read_csv("ca_demographics.tsv", sep="\t")
wa_dem = pd.read_csv("wa_demographics.tsv", sep="\t")

In [2]:
ca_enroll.head()

,local_student_id,student_id,site_name,last_name,first_name,grade
0,4462,195500,Everest Public High School,Eaton,Donna,12
1,4653,274335,Everest Public High School,Cortez,Mohammod,11
2,4496,207235,Everest Public High School,Hays,Makenzie,11
3,4436,185120,Everest Public High School,Guerra,Lleyton,12
4,4407,178785,Everest Public High School,Wills,Davey,12


In [3]:
wa_enroll.head()

,NAME,ID,STUDENT_NUMBER,LAST_NAME,FIRST_NAME,GRADE_LEVEL
0,Summit Public School: Atlas,2004,999999,Shenny,Howardo,12.0
1,Summit Public School: Atlas,2008,86805,Sheldon,Aadam,11.0
2,Summit Public School: Olympus,2012,80011,Andrew,Tony,12.0
3,Summit Public School: Atlas,2020,86676,Forster,Adeline,11.0
4,Summit Public School: Atlas,2024,86806,Cummings,Kali,8.0


In [4]:
ca_dem.head()

,student_id,gender,combined_race_ethnicity,has_iep
0,274220,M,Two or More Races,False
1,209085,M,White,False
2,208975,M,Hispanic,False
3,175055,F,Hispanic,False
4,207490,M,Hispanic,False


In [5]:
wa_dem.head()

,ID,IEP,RACE_ETHNICITY,GENDER
0,2004,0,Asian,X
1,2008,0,NaN,M
2,2012,0,NaN,M
3,2020,1,NaN,F
4,2024,0,NaN,F


In [6]:
# Next, let's join the California tables and the Washington tables.

cali = ca_enroll.merge(ca_dem, how="outer", on="student_id")

In [7]:
wash = wa_enroll.merge(wa_dem, how="outer", on="ID")

In [8]:
# Let's check for missing data.

cali.isnull().sum()

# Looks like a few missing entries...

local_student_id            0
student_id                  0
site_name                   0
last_name                   0
first_name                  0
grade                       0
gender                      1
combined_race_ethnicity    72
has_iep                    38
dtype: int64

In [9]:
cali[cali["gender"].isnull() == True]

# But no records missing from just one of the two tables.

,local_student_id,student_id,site_name,last_name,first_name,grade,gender,combined_race_ethnicity,has_iep
1178,25179,164220,Summit Public School: Denali,Pineda,Eddie,11,NaN,White,False


In [10]:
wash.isnull().sum()

# No records missing from the merge here either.
# A lot of race/ethnicity data is missing from the Washington set.

NAME                0
ID                  0
STUDENT_NUMBER      0
LAST_NAME           0
FIRST_NAME          0
GRADE_LEVEL        28
IEP                 0
RACE_ETHNICITY    844
GENDER              0
dtype: int64

In [11]:
# Let's try to use Student ID as the unique identifier
# Compare the column count with the number of unique values
# Just to ensure that each record has a unique Student ID

cali["student_id"].value_counts()

274430    1
268990    1
168635    1
244410    1
271510    1
         ..
222510    1
210220    1
267560    1
248005    1
245760    1
Name: student_id, Length: 3656, dtype: int64

In [12]:
# Washington's "STUDENT_NUMBER" field is shared
# Likely, one or more schools there are reusing old IDs.

wash["STUDENT_NUMBER"].value_counts()

86713    2
73515    2
87119    2
87121    2
87123    2
        ..
87020    1
87018    1
87017    1
87016    1
73728    1
Name: STUDENT_NUMBER, Length: 861, dtype: int64

In [13]:
wash[wash["STUDENT_NUMBER"] == 86713]

# Sure enough, Atlas is reusing IDs
# So Student ID alone won't work as an identifier

,NAME,ID,STUDENT_NUMBER,LAST_NAME,FIRST_NAME,GRADE_LEVEL,IEP,RACE_ETHNICITY,GENDER
233,Summit Public School: Atlas,3564,86713,James,Sana,11.0,0,NaN,F
290,Summit Public School: Atlas,4124,86713,Bate,Angharad,11.0,0,NaN,F


In [14]:
# Let's try using the ID field. Maybe that is already unique.

cali["local_student_id"].value_counts()

1399     5
1507     5
1467     5
1434     5
1453     5
        ..
25243    1
64471    1
25239    1
25237    1
36864    1
Name: local_student_id, Length: 3041, dtype: int64

In [15]:
cali[cali["local_student_id"] == 1399]

# Opposite to Washington, California is reusing Local IDs.

,local_student_id,student_id,site_name,last_name,first_name,grade,gender,combined_race_ethnicity,has_iep
426,1399,189625,Summit Preparatory Charter High School,Carver,Sonia,12,F,Hispanic,False
528,1399,189635,Summit Preparatory Charter High School,Traynor,Isma,12,F,Hispanic,True
563,1399,189640,Summit Preparatory Charter High School,Combs,Jarrad,12,M,Hispanic,False
816,1399,189645,Summit Preparatory Charter High School,Duke,Robyn,12,M,Hispanic,False
837,1399,189630,Summit Preparatory Charter High School,Greenwood,Sofia,12,M,Hispanic,False


In [16]:
# Let's try concatenating Local ID and Student ID. This will be
# almost guaranteed to generate a unique number for each student.

cali["ID_SN"] = cali["local_student_id"].astype(str) + \
                cali["student_id"].astype(str)
cali["ID_SN"] = cali["ID_SN"].astype('int64')

In [17]:
wash["ID_SN"] = wash["ID"].astype(str) + wash["STUDENT_NUMBER"].astype(str)
wash["ID_SN"] = wash["ID_SN"].astype('int64')

In [18]:
# Check again for uniqueness

cali["ID_SN"].value_counts()

50199269370    1
50271269615    1
16954215140    1
49427192550    1
4592252225     1
              ..
25154164100    1
36917222785    1
37217269120    1
4496207235     1
16797188100    1
Name: ID_SN, Length: 3656, dtype: int64

In [19]:
wash["ID_SN"].value_counts()

# Success! We have a unique identifier for each student

1167486975    1
1872787141    1
470080155     1
210186909     1
1924580324    1
             ..
521680221     1
715273564     1
1340680258    1
1390873764    1
218486785     1
Name: ID_SN, Length: 1141, dtype: int64

In [22]:
# Concatenate the two DataFrames into one.
# To do this, we'll need to rename the columns so they are matching.

wash = wash.rename(columns={"LAST_NAME": "last_name", \
                                   "FIRST_NAME": "first_name", \
                                   "NAME": "site_name", \
                                   "GRADE_LEVEL": "grade", \
                                   "RACE_ETHNICITY": "combined_race_ethnicity", \
                                   "GENDER": "gender", \
                                   "IEP": "has_iep"})

wash["has_iep"] = wash["has_iep"].astype(bool)

In [23]:
wash.head()

,site_name,ID,STUDENT_NUMBER,last_name,first_name,grade,has_iep,combined_race_ethnicity,gender,ID_SN
0,Summit Public School: Atlas,2004,999999,Shenny,Howardo,12.0,False,Asian,X,2004999999
1,Summit Public School: Atlas,2008,86805,Sheldon,Aadam,11.0,False,NaN,M,200886805
2,Summit Public School: Olympus,2012,80011,Andrew,Tony,12.0,False,NaN,M,201280011
3,Summit Public School: Atlas,2020,86676,Forster,Adeline,11.0,True,NaN,F,202086676
4,Summit Public School: Atlas,2024,86806,Cummings,Kali,8.0,False,NaN,F,202486806


In [24]:
# Now drop columns we don't need anymore

cali = cali[["ID_SN", "last_name", "first_name", "site_name", "grade", \
             "combined_race_ethnicity", "gender", "has_iep"]]

wash = wash[["ID_SN", "last_name", "first_name", "site_name", "grade", \
             "combined_race_ethnicity", "gender", "has_iep"]]

In [25]:
# Lastly, concatenate the two.

combined = pd.concat([cali, wash], axis=0)

In [26]:
# Double check to make sure all records made it.

combined.count() == cali.count() + wash.count()

ID_SN                      True
last_name                  True
first_name                 True
site_name                  True
grade                      True
combined_race_ethnicity    True
gender                     True
has_iep                    True
dtype: bool

In [28]:
# Export for submission

combined.to_csv("output.tsv", sep="\t")